In [1]:
import os.path as op
import os
import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
data_dirs = ['../data/results_D100', '../data/results_D25']
files = [op.join(d, f)
         for d in data_dirs
         for f in os.listdir(d)
         if f.endswith('csv')]

In [3]:
full_results = []
for fl in files:
    tmpres = pd.read_csv(fl, index_col=0)
    fname_pieces = fl.split('_')
    dset = fname_pieces[1].split('/')[0]
    experiment = fname_pieces[2]
    n_mca = int(fname_pieces[3])
    clf = fname_pieces[-4]
    dimens = fname_pieces[-2]
    preproc = fname_pieces[-3]
    random_seed = fname_pieces[-1].strip('.csv')
    resdict = {
        "Classifier": clf,
        "Experiment": experiment,
        "Simulations": n_mca,
        "Preprocessing": preproc,
        "Dimensionality": dimens,
        "Dataset": dset,
        "rs": random_seed
    }

    for idx, r in tmpres.iterrows():
        resdict_copy = deepcopy(resdict)
        # Only add ref performance for the "full" dataset
        if r.aggregation == "ref" and (n_mca != 20 or experiment != 'mca'):
            continue
        for k, v in r.items():
            resdict_copy[k] = v
        full_results += [resdict_copy]
        del resdict_copy

df_results = pd.DataFrame.from_dict(full_results)

In [4]:
n_pre = 4
n_dim = 2
n_clf = 5
n_agg_m = 6
n_agg_s = 4
n_mca = 9
n_ss = 2

n_pre*n_dim*n_clf*n_agg_m*n_mca + n_pre*n_dim*n_clf

n_pre*n_dim*n_clf*n_agg_m*n_mca + n_pre*n_dim*n_clf + n_pre*n_dim*n_clf*n_agg_s*n_ss

2520

In [5]:
df_results.query('Dataset == "D100"')

,Classifier,Experiment,Simulations,Preprocessing,Dimensionality,Dataset,rs,aggregation,acc,f1,p_acc,p_f1,test_acc,test_f1,p_test_acc,p_test_f1,test_mean_acc,test_mean_f1,n_models
0,ada,mca,14,zgraph,fa,D100,41,truncate,0.640833,0.736623,0.0862,0.10300,0.750000,0.827586,0.008,0.003,0.670000,0.777266,5
1,svc,mca,14,loggraph,fa,D100,41,median,0.666667,0.769231,0.0818,0.07680,0.700000,0.823529,0.023,0.005,0.670000,0.795820,5
2,rfc,mca,16,graph,fa,D100,41,mega,0.755690,0.836674,0.0000,0.00000,0.711765,0.825623,0.000,0.000,0.735294,0.832746,5
3,lrc,mca,18,loggraph,fa,D100,41,mean,0.550833,0.644421,0.3588,0.34000,0.600000,0.692308,0.131,0.159,0.630000,0.721086,5
4,svc,mca,10,rankgraph,fa,D100,41,truncate,0.666667,0.774945,0.0780,0.06500,0.700000,0.823529,0.021,0.012,0.660000,0.790303,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,knn,mca,2,rankgraph,pca,D100,41,none,0.718833,0.826089,0.0260,0.01722,0.650000,0.787879,0.060,0.027,0.668000,0.798788,50
2556,knn,mca,2,rankgraph,pca,D100,41,meta,0.679167,0.787066,0.0540,0.04640,0.650000,0.787879,0.060,0.027,0.660000,0.776502,5
2557,knn,mca,18,zgraph,pca,D100,41,mean,0.719167,0.801591,0.0692,0.07620,0.750000,0.838710,0.008,0.010,0.740000,0.828341,5
2558,lrc,mca,20,graph,fa,D100,41,mean,0.718333,0.773141,0.0590,0.09240,0.700000,0.785714,0.018,0.021,0.660000,0.744184,5


In [6]:
df_results.to_csv("../data/aggregation_results.csv", index=False)